<a href="https://www.kaggle.com/code/drkaggle22/toxic-comments-classification-tf-idf-nltk?scriptVersionId=176525522" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [2]:
df_train = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip', index_col='id')

df_test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip', index_col='id')

In [3]:
pd.set_option('display.max_rows', None)

In [4]:
df = pd.concat([df_train, df_test])

In [5]:
df_train['comment_text'] = df_train['comment_text'].str.lower()
df_test['comment_text'] = df_test['comment_text'].str.lower()

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159571 entries, 0000997932d777bf to fff46fc426af1f9a
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   comment_text   159571 non-null  object
 1   toxic          159571 non-null  int64 
 2   severe_toxic   159571 non-null  int64 
 3   obscene        159571 non-null  int64 
 4   threat         159571 non-null  int64 
 5   insult         159571 non-null  int64 
 6   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 9.7+ MB


In [7]:
import re

def remove_special_characters(text):
    text = re.sub(r'http\S+', ' ', text )
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\bhttps?://[a-zA-Z0-9-]+(?:\.[a-zA-Z0-9-]+)+\b', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\d', ' ', text)  # Corrected line
    text= re.sub(r'[\u4e00-\u9fff]+', ' ', text)
    return text

df_train['comment_text'] = df_train['comment_text'].apply(remove_special_characters)
df_test['comment_text'] = df_test['comment_text'].apply(remove_special_characters)

print(df_train['comment_text'].head(100))


id
0000997932d777bf    explanation why the edits made under my userna...
000103f0d9cfb60f    d aww he matches this background colour i m se...
000113f07ec002fd    hey man i m really not trying to edit war it s...
0001b41b1c6bb37e    more i can t make any real suggestions on impr...
0001d958c54c6e35    you sir are my hero any chance you remember wh...
00025465d4725e87    congratulations from me as well use the tools ...
0002bcb3da6cb337         cocksucker before you piss around on my work
00031b1e95af7921    your vandalism to the matt shirvington article...
00037261f536c51d    sorry if the word nonsense was offensive to yo...
00040093b2687caa    alignment on this subject and which are contra...
0005300084f90edc    fair use rationale for image wonju jpg thanks ...
00054a5e18b50dd4    bbq be a man and lets discuss it maybe over th...
0005c987bdfc9d4b    hey what is it talk what is it an exclusive gr...
0006f16e4e9f292e    before you start throwing accusations and warn...
00070ef96486d6f9 

In [8]:
import string 
from nltk import word_tokenize

df_train['tokens'] = df_train['comment_text'].apply(word_tokenize)
df_test['tokens'] = df_test['comment_text'].apply(word_tokenize)


In [9]:
df_train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,tokens
id,,,,,,,,
0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0,"[explanation, why, the, edits, made, under, my..."
000103f0d9cfb60f,d aww he matches this background colour i m se...,0,0,0,0,0,0,"[d, aww, he, matches, this, background, colour..."
000113f07ec002fd,hey man i m really not trying to edit war it s...,0,0,0,0,0,0,"[hey, man, i, m, really, not, trying, to, edit..."
0001b41b1c6bb37e,more i can t make any real suggestions on impr...,0,0,0,0,0,0,"[more, i, can, t, make, any, real, suggestions..."
0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0,"[you, sir, are, my, hero, any, chance, you, re..."


In [10]:
X = df_train['comment_text']  # Assuming 'comment_text' contains your text data
y = df_train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=42)
# Define TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2),
                             min_df=3,
                             max_df=0.9,
                             strip_accents='unicode',
                             use_idf=True,
                             smooth_idf=True,
                             sublinear_tf=True,
                             stop_words='english')

# Concatenate training and validation data to fit the vectorizer
X_concat = pd.concat([X_train, X_valid])

# Fit the vectorizer on concatenated data
vectorizer.fit(X_concat)

# Transform concatenated data
X_concat_tfidf = vectorizer.transform(X_concat)

# Split the concatenated data back into training and validation parts
X_train_tfidf, X_valid_tfidf = X_concat_tfidf[:len(X_train)], X_concat_tfidf[len(X_train):]

# Print the shapes of transformed data for confirmation
print("X_train_tfidf shape:", X_train_tfidf.shape)
print("X_valid_tfidf shape:", X_valid_tfidf.shape)

# Now you can proceed to split the labels accordingly
y_concat = pd.concat([y_train, y_valid])
y_train, y_valid = y_concat[:len(X_train)], y_concat[len(X_train):]


X_train_tfidf shape: (127656, 264325)
X_valid_tfidf shape: (31915, 264325)


In [12]:
x = X_train_tfidf
val_x = X_valid_tfidf

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer


# Initialize MultiLabelBinarizer with the classes
mlb = MultiLabelBinarizer()

# Transform target labels into binary form
y_train_binary = mlb.fit_transform(y_train)
y_valid_binary = mlb.transform(y_valid)

# At this point, y_train_binary and y_valid_binary are ready to be used in your model training


In [14]:
import numpy as np

# List of class labels
class_labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Initialize empty arrays to store binary labels
y_train_binary = np.zeros((len(y_train), len(class_labels)), dtype=int)
y_valid_binary = np.zeros((len(y_valid), len(class_labels)), dtype=int)

# Manually create binary labels for each class
for i, label in enumerate(class_labels):
    y_train_binary[:, i] = y_train[label]
    y_valid_binary[:, i] = y_valid[label]

# At this point, y_train_binary and y_valid_binary are ready to be used in your model training


In [15]:
print("y_train_binary shape:", y_train_binary.shape)
print("y_valid_binary shape:", y_valid_binary.shape)


y_train_binary shape: (127656, 6)
y_valid_binary shape: (31915, 6)


In [16]:
print("X_train_tfidf shape:", X_train_tfidf.shape)
print("y_train_binary shape:", y_train_binary.shape)


X_train_tfidf shape: (127656, 264325)
y_train_binary shape: (127656, 6)


In [17]:
from sklearn.metrics import f1_score  # Import F1-score for multi-label evaluation
from sklearn.neural_network import MLPClassifier
# Number of epochs
epochs = 20
learning_rate = 0.001

# Initialize MLPClassifier with verbose=True to display progress during training
clf = MLPClassifier(hidden_layer_sizes=(20, 10), max_iter=epochs, activation='relu',
                   solver='adam', random_state=42, verbose=True, learning_rate_init=learning_rate)

# Fit the classifier with the training data and binary target labels
clf.fit(X_train_tfidf, y_train_binary)

# Predict probabilities for validation data
y_valid_pred = clf.predict(X_valid_tfidf)

# Calculate macro F1-score for multi-label evaluation
macro_f1 = f1_score(y_valid_binary, y_valid_pred, average='macro')
print("Validation Macro F1-score:", macro_f1)

# Additionally, calculate micro F1-score for another perspective
micro_f1 = f1_score(y_valid_binary, y_valid_pred, average='micro')
print("Validation Micro F1-score:", micro_f1)


Iteration 1, loss = 1.36404451
Iteration 2, loss = 0.35632148
Iteration 3, loss = 0.24481085
Iteration 4, loss = 0.19213450
Iteration 5, loss = 0.16175038
Iteration 6, loss = 0.14099514
Iteration 7, loss = 0.12400084
Iteration 8, loss = 0.10906771
Iteration 9, loss = 0.09517811
Iteration 10, loss = 0.08308925
Iteration 11, loss = 0.07238356
Iteration 12, loss = 0.06255375
Iteration 13, loss = 0.05474718
Iteration 14, loss = 0.04833241
Iteration 15, loss = 0.04400711
Iteration 16, loss = 0.04071312
Iteration 17, loss = 0.03791521
Iteration 18, loss = 0.03621378
Iteration 19, loss = 0.03480324
Iteration 20, loss = 0.03241392
Validation Macro F1-score: 0.5111717647059636


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Validation Micro F1-score: 0.6815100154083205


In [18]:
X_test_tokens = df_test['comment_text']

# Transform the test data using the fitted TF-IDF vectorizer
X_test_tfidf = vectorizer.transform(X_test_tokens)

# Print the shape of transformed test data for confirmation
print("X_test_tfidf shape:", X_test_tfidf.shape)

X_test_tfidf shape: (153164, 264325)


In [19]:
# Assuming clf is the trained MLPClassifier model

# Predict probabilities for test data
y_test_prob = clf.predict_proba(X_test_tfidf)

# Print the shape of predicted probabilities for confirmation
print("Shape of predicted probabilities:", y_test_prob.shape)

# Now you have predicted probabilities for each class for the test data


Shape of predicted probabilities: (153164, 6)


In [20]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [21]:
# Convert the predicted probabilities to float
y_test_prob_float = y_test_prob.astype(float)

# Print the updated type of predicted probabilities for confirmation
print("Type of predicted probabilities:", y_test_prob_float.dtype)

# Now y_test_prob_float contains predicted probabilities for each class for each data point in the test data, stored as float numbers


Type of predicted probabilities: float64


In [22]:
import numpy as np

# Assuming y_test_prob_float is your array of predicted probabilities

# Set numpy printing options to suppress scientific notation
np.set_printoptions(suppress=True)

# Print the updated predicted probabilities
print(y_test_prob_float)


[[1.         0.44677023 0.99999597 0.00002    0.99988847 0.01466082]
 [0.00015131 0.         0.00000307 0.         0.00000581 0.        ]
 [0.00213721 0.         0.0001751  0.00000012 0.00016436 0.00000006]
 ...
 [0.00000568 0.         0.0000008  0.         0.00000064 0.        ]
 [0.00004993 0.         0.00000022 0.         0.00000673 0.00000002]
 [0.99999944 0.00002397 0.98707946 0.00000003 0.8060696  0.00000152]]


In [23]:
# Set the floating-point format for display
pd.options.display.float_format = '{:.6f}'.format

# Create submission DataFrame
submid = pd.DataFrame({'id': df_test.index})  # Use index as 'id' column
submission = pd.concat([submid, pd.DataFrame(y_test_prob_float, columns=classes)], axis=1)

# Write submission DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)


In [24]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,1.000000,0.446770,0.999996,0.000020,0.999888,0.014661
1,0000247867823ef7,0.000151,0.000000,0.000003,0.000000,0.000006,0.000000
2,00013b17ad220c46,0.002137,0.000000,0.000175,0.000000,0.000164,0.000000
3,00017563c3f7919a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,00017695ad8997eb,0.000000,0.000000,0.000001,0.000000,0.000036,0.000000
